In [ ]:
!pip install torch
!pip install tokenizers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 29.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 32.3 MB/s 
     |████████████████████████████████| 163 kB 64.2 MB/s 


In [ ]:
vocab_length = 30522

In [ ]:
import tokenizers
# 
roman_BWPT = tokenizers.BertWordPieceTokenizer(
#     # add_special_tokens=True, # This argument doesn't work in the latest version of BertWordPieceTokenizer
     unk_token='[UNK]',
     sep_token='[SEP]',
     cls_token='[CLS]',
     clean_text=True,
     handle_chinese_chars=True,
     strip_accents=True,
     lowercase=True,
     wordpieces_prefix='##'
 )
 
roman_BWPT.train(
     files=["roman.txt"],
     vocab_size=vocab_length,
     min_frequency=3,
     limit_alphabet=1000,
     show_progress=True,
     special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
 )
 
roman_BWPT.save_model(".", "roman-urdu"+str(vocab_length))

['./roman-urdu30522-vocab.txt']

In [ ]:
# Load the tokenizer
from transformers import BertTokenizer, LineByLineTextDataset

vocab_file_dir = 'vocab.txt' 

tokenizer = BertTokenizer.from_pretrained(vocab_file_dir)

sentence = 'ppp chill pmln lorey lag gayi har tarah'

encoded_input = tokenizer.tokenize(sentence)
print(encoded_input)
# print(encoded_input['input_ids'])

['pp', '##p', 'chill', 'pm', '##ln', 'lore', '##y', 'la', '##g', 'gay', '##i', 'ha', '##r', 'tara', '##h']


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1682: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [ ]:
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = '/content/roman.txt',
    block_size = 128  # maximum sequence length
)

print('No. of lines: ', len(dataset)) # No of lines in your datset

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


No. of lines:  87883


In [ ]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=50000,
    hidden_size=768, 
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=512
)
 
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

No of parameters:  81965648


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_dir',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    #prediction_loss_only=False,
)

In [ ]:
%%time
trainer.train()
trainer.save_model('model_dir')

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 87883
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2747


Step,Training Loss
500,7.259600
1000,6.414300
1500,6.182900
2000,6.061700
2500,5.979100




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to model_dir
Configuration saved in model_dir/config.json
Model weights saved in model_dir/pytorch_model.bin


CPU times: user 17min 45s, sys: 4.16 s, total: 17min 49s
Wall time: 17min 58s


In [ ]:
from transformers import pipeline

model = BertForMaskedLM.from_pretrained('model_dir')

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

loading configuration file model_dir/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading weights file model_dir/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at model_dir.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForMaskedLM for pr

In [ ]:
fill_mask(' [MASK] ka shukar pti won')

[{'score': 0.031184498220682144,
  'token': 16455,
  'token_str': 'a l l a h',
  'sequence': 'allah ka shukar pti won'},
 {'score': 0.019820570945739746,
  'token': 9706,
  'token_str': 'a p',
  'sequence': 'ap ka shukar pti won'},
 {'score': 0.019775265827775,
  'token': 1038,
  'token_str': 'b',
  'sequence': 'b ka shukar pti won'},
 {'score': 0.018191535025835037,
  'token': 6300,
  'token_str': 'y e',
  'sequence': 'ye ka shukar pti won'},
 {'score': 0.01712663099169731,
  'token': 2909,
  'token_str': 's i r',
  'sequence': 'sir ka shukar pti won'}]

In [ ]:
#

In [ ]:
#

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 87883
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2747


Step,Training Loss


Step,Training Loss


KeyboardInterrupt: ignored